In [1]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

## Import JSON 

In [8]:
zip_dir  = "../data/zip2"

files = glob(os.path.join(zip_dir, '*'))
for filename in files:
    if '.zip' in filename:
        pass
    elif '.json' in filename:
        pass
    else:
        newfile = filename+'.json'
        os.rename(filename, newfile)

jsons = glob(os.path.join(zip_dir, '*.json'))

In [9]:
# jsons = jsons[:2]
jsons

['../data/zip2/part-v003-o001-r-00000.json',
 '../data/zip2/part-v003-o001-r-00001.json',
 '../data/zip2/part-v003-o001-r-00002.json',
 '../data/zip2/part-v003-o001-r-00003.json',
 '../data/zip2/part-v003-o001-r-00004.json',
 '../data/zip2/part-v003-o001-r-00005.json',
 '../data/zip2/part-v003-o001-r-00006.json',
 '../data/zip2/part-v003-o001-r-00007.json',
 '../data/zip2/part-v003-o001-r-00008.json',
 '../data/zip2/part-v003-o001-r-00009.json']

## Add location data

In [10]:
# jsons = ['../data/part_v003_o001_r_00000.json', ..., '../data/part_v003_o001_r_00001.json']

dfrts = []
dfors = []
for i, json in enumerate(jsons):
    # Instance Preparation
    dates = []
    tweets = []
    user_ids = []
#     rts = []

#     # tweet's location
#     place_tweet = []
    # tweet's location
    place_fullname_tweet = []
    # tweet's country
    nationality = []
    
    start = time.time()
    # Load json file: date and tweet
    with open(json, 'r', encoding='utf8') as file:
        pet_parse = ijson.parse(file, multiple_values=True)
        for prefix, event, value  in pet_parse:
            # Date
            if prefix == 'created_at':
                dates.append(datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y'))
            # Tweet 
            if prefix == 'text':
                tweets.append(value.replace('\n', '').replace('\t', '').replace('\r', '').replace('\r\n', '').replace('　', '')) # Delte space and indet and \r
            # User id 
            if prefix == 'user.id':
                user_ids.append(value)
                
#             # RT Flag
#             if len(dates)-1 == len(rts) and prefix == 'retweeted_status':
#                 rts.append(True)
#             if len(dates)-2 == len(rts):
#                 rts.append(False)
            
#             # tweet's location
#             if prefix == 'place.name':
#                 place_tweet.append(value)
#             if len(dates)-2 == len(place_tweet):
#                 place_tweet.append('nan')

            # tweet's location - fullname
            if prefix == 'place.full_name':
                place_fullname_tweet.append(value)
            if len(dates)-2 == len(place_fullname_tweet):
                place_fullname_tweet.append('nan')
                
            # tweet's nationality
            if prefix == 'place.country':
                nationality.append(value)
            if len(dates)-2 == len(nationality):
                nationality.append('nan')

        # Add missing data
#         if len(dates) != len(rts):
#                 rts.append(False)
        if len(dates) != len(place_fullname_tweet):
                place_fullname_tweet.append('nan')
        if len(dates) != len(nationality):
                nationality.append('nan')

    print("Loading json elapsed_time:{0}".format(time.time() - start) + "[sec]")
    
#     print(len(dates), len(tweets), len(user_ids), len(rts), len(nationality), len(place_tweet))
    
    # Create dataframe
    data = np.vstack([user_ids, dates, tweets, nationality, place_fullname_tweet]).T
    df = pd.DataFrame(data, columns=['user_id', 'date', 'tweet', 'nationality', 'place_tweet'])
    
    import datetime as dt
    
    # Df only with location data
    df = df[df.place_tweet != 'nan']
    
    # Change time zone
    df['date'] = df['date'] + dt.timedelta(hours = 9)
    
    # Date
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    df['time'] = df['date'].dt.time
    df['date'] = df['date'].dt.date
    
#     # divide df by RT_flag 
#     dfrt = df[df['RT_flag'] == True]
#     dfor = df[df['RT_flag'] == False]
    
    # Save to csv file
    df.to_csv('../data/dfs_location2/df_lct'+str(i)+'.csv')
#     dfrt.to_csv('../data/dfs_location/dfrt_lct'+str(i)+'.csv')
    
    # Append to dfs
#     dfrts.append(dfrt)
#     dfors.append(dfor)

Loading json elapsed_time:718.873034954071[sec]
Loading json elapsed_time:648.138466835022[sec]
Loading json elapsed_time:526.3302919864655[sec]
Loading json elapsed_time:507.3329060077667[sec]
Loading json elapsed_time:496.4747657775879[sec]
Loading json elapsed_time:536.7752206325531[sec]
Loading json elapsed_time:555.0317189693451[sec]
Loading json elapsed_time:566.3812553882599[sec]
Loading json elapsed_time:529.7259113788605[sec]
Loading json elapsed_time:740.5443599224091[sec]


In [107]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 


## Assign area 

In [196]:
# read csv of dictionary
def crate_area_dictionary(file):
    area = pd.read_csv(file)
    area_jp = area.drop(['pref_en'], axis=1).set_index(['pref_jp']).Area.to_dict()
    area_en = area.drop(['pref_jp'], axis=1).set_index(['pref_en']).Area.to_dict()
    return area_jp, area_en

area_jp, area_en = crate_area_dictionary('../data/area.csv')
area_shutoken_jp, area_shutoken_en = crate_area_dictionary('../data/area_shutoken.csv')
area_tokyo_jp, area_tokyo_en = crate_area_dictionary('../data/area_tokyo.csv')


def df_by_area(df, d_area_jp, d_area_en):
    l_area = []
    for natio, place in zip(df.nationality, df.place_tweet):
        if natio == '日本' and place.split(' ')[0] in d_area_jp:
            l_area.append(d_area_jp[place.split(' ')[0]])
        elif natio == '日本' and place.split(' ')[-1] in d_area_jp:
            l_area.append(d_area_jp[place.split(' ')[-1]])
        elif l_area == 'Japan' and place.split(', ')[-1] in d_area_en:
            l_area.append(d_area_en[place.split(', ')[-1]])
        elif natio == 'Japan' and place.split(', ')[0] in d_area_en:
            l_area.append(d_area_en[place.split(', ')[0]])
        else:
            l_area.append('nan')
    df['area'] = l_area
    df_lct = df[df.area != 'nan']
    return df_lct

## read csv

In [197]:
# Read ORs
zip_dir  = "../data/dfs_location2" # new_dfs2_jst
csvfiles = glob(os.path.join(zip_dir, 'df*'))

dfs = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfs.append(df)

In [198]:
df_lcts = [df_by_area(df, area_jp, area_en)  for df in dfs]

## Analyze data monthly

In [199]:
# Concat dfs
for i, df in enumerate(df_lcts):
    if i == 0:
        dfm = df
    else:
        dfm = pd.concat([dfm, df], axis=0)
_df = dfm.copy()
df_shutoken = df_by_area(_df, area_shutoken_jp, area_shutoken_en)
dfm = pd.concat([dfm, df_shutoken], axis=0)
df_tokyo = df_by_area(_df, area_tokyo_jp, area_tokyo_en)
dfm = pd.concat([dfm, df_tokyo], axis=0)
# Drop axis
dfm = dfm.drop(['nationality', 'place_tweet'], axis=1)

## group by area

In [200]:
area_dict = {}
areas = dfm.area.unique().tolist()
initial_area = ['A','B','C','D','E','F','G','H','I','J']
for area, initial in zip(areas, initial_area):
    area_dict[area] = initial

In [201]:
# Save into csv of each area
for area in areas:
    dfm.groupby('area').get_group(area).to_csv('../data/dfs_by_area2/mar_'+area_dict[area]+'.csv')

In [176]:
dfm

,user_id,date,tweet,year,month,day,hour,time,area
0,43809995,2020-02-29,@rihito_karube お忙しいのにお返事ありがとうございますそろそろ社内で政治部とガ...,2020,2,29,23,23:59:52,関東地方
1,949838786287906817,2020-02-29,バカは言い過ぎですが、なんの根拠もないデマに踊らさせて…こういうのを付和雷同って言います。ち...,2020,2,29,23,23:58:47,中国地方
2,925958009381036032,2020-02-29,@piyo_0126 GPS撮影会の井の頭公園の桜撮影🌸に申し込みました。まだ一度も抽選に当...,2020,2,29,23,23:58:41,関東地方
4,143131975,2020-02-29,@mugithecat いいよ。いいよ。俺も久しぶりの関西遠征楽しみにしてたのに家でぐだぐだ...,2020,2,29,23,23:58:04,関東地方
5,3312098114,2020-02-29,#コロナウイルスばかりでウンザリなので山電の画像を貼る6002＋6003の直通特急が懐かしい...,2020,2,29,23,23:57:20,近畿地方
...,...,...,...,...,...,...,...,...,...
3702,3050839291,2020-02-01,@kent711hatogaya ラピアリバレルめちゃめちゃ怠いですよ😅,2020,2,1,0,00:15:37,東京
3703,2924259060,2020-02-01,@ro_ktn 変な人もいるから気を付けてよ～🤔琴っちゃん可愛いんだから夜道心配だよ……😩,2020,2,1,0,00:14:30,東京
3705,197788314,2020-02-01,業者さんが泣きそうなくらいガッカリして帰っていった。だって、教えてくれなかったから直ぐに予定...,2020,2,1,0,00:12:39,東京
3712,93646446,2020-02-01,犠牲者には悪いのだけど、読ませる手記だなあ。そして、警察はこれ読んで惨めにならないのかね？自...,2020,2,1,0,00:09:40,東京


In [205]:
# Read ORs
zip_dir  = "../data/dfs_by_area2" # new_dfs2_jst
csvfiles = glob(os.path.join(zip_dir, 'mar_*'))

feb_dfs = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop(['Unnamed: 0', 'area'], axis=1)
    feb_dfs.append(df)
    break

In [207]:
feb_dfs[0]

,user_id,date,tweet,year,month,day,hour,time
0,778167031598374912,2020-03-31,なんかもうだるい自分が嫌,2020,3,31,23,23:59:58
1,2989602496,2020-03-31,早く!!対応してください！野党やら与党やら言ってる場合ではないですよ！亡くなった方を持ちだし...,2020,3,31,23,23:59:55
2,2300772428,2020-03-31,とーやま校長10年間お疲れ様でした私の青春時代には必ず貴方がいました音楽と夢への活力を与えて...,2020,3,31,23,23:57:53
3,235017720,2020-03-31,娘や娘と仲良くしてくれたお友達にはこんな時で不安だろうけど明日からの社会人生活頑張ってほしい...,2020,3,31,23,23:44:35
4,1188641708751212544,2020-03-31,明日から年度始めだ忙しくなるよ😢何よりコロナが気がかりだ😵,2020,3,31,23,23:36:05
...,...,...,...,...,...,...,...,...
2128,604331616,2020-03-01,「本当に私でも副収入をゲットできるの？」誰でも最初は不安ですよね。自分にもダウンさんにも無理...,2020,3,1,1,01:56:32
2129,604331616,2020-03-01,「本当に私でも副収入をゲットできるの？」誰でも最初は不安ですよね。自分にもダウンさんにも無理...,2020,3,1,1,01:14:45
2130,3592667114,2020-03-01,ダルい、し、もう良いや、開始へ自動へ,2020,3,1,1,01:00:03
2131,97611577,2020-03-01,@xHaxrgiqIgj4DRK @sukai4055 使いかけだと売られているものまであり...,2020,3,1,0,00:22:28


## test

In [209]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfors, filetype, savedir, key_lang, area):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    
    # Keyword 
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   

        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month', 'user_id', 'day'],axis=1)
        
            # Save csv
            outname = 'area'+area+'_'+filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
#                 existedfile = pd.read_csv(savename)
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)
                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

In [210]:
# Run
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['A', 'A', 'V', 'F', 'C']
for area in initial_area:
    for filetype in filetypes:
        findtwt_hour(feb_dfs, filetype=filetype, savedir='../results_location2/hour', key_lang='jp', area=area)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 1/1 [00:00<00:00,  7.20it/s]


## Read csv

In [17]:
# Read ORs
zip_dir  = "../data/dfs" # new_dfs2_jst
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfors.append(df)

In [7]:
# Read Rts
zip_dir  = "../data/new_dfs2_jst"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df1 = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfors.append(df1)

## Check csv data

In [5]:
for df in dfors:
    print(df.RT_flag.unique())

[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]


In [5]:
for df in dfrts:
    print(df.RT_flag.unique())

[True]
[True]
[True]
[True]
[True]
[True]
[True]
[True]
[True]
[True]


## df info

In [7]:
c = [len(df) for df in dfors]
print('The amount of tweets included RT:', sum(c))

The amount of tweets included RT: 9286476


In [34]:
c = [len(df) for df in dfrts]
print('The amount of tweets included RT:', sum(c))

The amount of tweets included RT: 6692071


## Create csv by day

In [65]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_day(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
            
    # Find tweets
    for dfor in tqdm(dfors):
         # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        # Add columns of keywords whose cell have 1 if this tweet includes a keyword 
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True) 
        dfs.append(dft)
        
    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    # Save
    outname = filetype+'_original.csv'
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    savename = os.path.join(outdir, outname)
    dfnew.to_csv(savename)

    return dfnew


## Run day

In [66]:
# OR
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfors, filetype=filetype, savedir='../new_results_jst/orjp/day', key_lang='jp') # new_results_jst2

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:29<00:00,  8.92s/it]


In [81]:
# OR
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfors, filetype=filetype, savedir='../new_results_jst/oren/day', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:13<00:00,  1.31s/it]


In [11]:
# RT
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfrts, filetype=filetype, savedir='../new_results_jst/rtjp/day', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [03:53<00:00, 23.33s/it]


In [67]:
# RT
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfrts, filetype=filetype, savedir='../new_results_jst/rten/day', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:17<00:00,  1.74s/it]


## Create csv by hour 

In [10]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    
    # Keyword 
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   

        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month', 'user_id', 'RT_flag', 'day'],axis=1)
            
            # Save csv
            outname = filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
                existedfile = pd.read_csv(savename)
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)
                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

## Run Orjp, Oren, Rtjp, Rten

In [11]:
# Orjp
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='../new_results_jst2/orjp/hour', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:31<00:00,  9.11s/it]


In [88]:
# Oren
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='../new_results_jst2/oren/hour', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:12<00:00,  1.30s/it]


In [100]:
# Rtjp
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfrts, filetype=filetype, savedir='../new_results_jst2/rtjp/hour', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:56<00:00, 11.65s/it]


In [101]:
# Rten
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfrts, filetype=filetype, savedir='../new_results_jst2/rten/hour', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:17<00:00,  1.75s/it]


## Check 

In [77]:
dft = pd.read_csv('../results/rtjp/hour/T2020-02-28.csv')
dft.T1.sum()

7827

In [ ]:
	T1
21	2958
22	2637
23	2546
24	4883
25	3352
26	2553
27	3206
28	5117
29	7827
30	2925

## コロナ含有率

In [50]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_corona(dfors, filetype="T"):
    # Prepration of keywords
    ## コロナ専用
    keywords = ['コロナ']
    
    c = [] # count volume
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        c.append(len(dfor_c))
    
    print(sum(c), 'tweets conatin "コロナ"')

# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
findtwt_corona(dfrts, filetype="T")
          

100%|██████████| 10/10 [00:04<00:00,  2.15it/s]

963821 tweets conatin "コロナ"


## Corona by day

In [102]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwtCrn_day(dfors, savename, savedir):
    
    dfs = []

    # Keyowrds
    keywords = ['コロナ', 'Corona', 'corona']
    filenames = ['コロナ', 'Corona', 'corona']

    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet include keywords
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True)
        
        dfs.append(dft)

    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    # Save
    outname = savename+'.csv'
    outdir = savedir

    if not os.path.exists(outdir):
            os.makedirs(outdir)

    savename = os.path.join(outdir, outname)    
    dfnew.to_csv(savename)

    return dfnew

In [94]:
dfnew = findtwtCrn_day(dfors, savename='corona', savedir='../new_results_jst/Corona/or')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


In [103]:
dfnew = findtwtCrn_day(dfrts, savename='corona', savedir='../new_results_jst2/Corona/rt')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


## Corona by hour

In [91]:
def findtwtCrn_hour(dfors, savename, savedir):
    # Prepartion 
    results = []
    dfs = []
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    
    # Keyowrds
    keywords = ['コロナ', 'Corona', 'corona']
    filenames = ['コロナ', 'Corona', 'corona']
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    filetype = savename
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   
        
        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month', 'user_id', 'RT_flag', 'day'],axis=1)
            
            outname = filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
                existedfile = pd.read_csv(savename)
                
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)

                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

In [92]:
findtwtCrn_hour(dfors, savename='corona', savedir='../new_results_jst2/Corona/or')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


In [104]:
findtwtCrn_hour(dfrts, savename='corona', savedir='../new_results_jst/Corona/rt')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:15<00:00,  1.54s/it]


In [115]:
t1 = pd.read_csv('../new_results_jst/rten/hour/V2020-02-01.csv')
t2 = pd.read_csv('../new_results/rten/hour/V2020-01-31.csv')


In [117]:
print(t1.loc[0:8,'V4'].sum(), t2.V4.sum())

0 0


In [114]:
dfrts[0].head()

,user_id,date,tweet,RT_flag,year,month,day,hour,time
0,1111855071380168705,2020-03-31,RT @k7LssPYI5D85fxI: https://t.co/5KgwBN00NS志村...,True,2020,3,31,23,23:59:58
1,127471475,2020-03-31,RT @84MadokaMary: この国には知らんだけでめちゃめちゃな量の社会保障があるの...,True,2020,3,31,23,23:59:58
2,138821520,2020-03-31,RT @zibumitunari: 明日はエイプリルフールだが、コロナ関連の嘘は絶対にやめよ...,True,2020,3,31,23,23:59:58
3,962192359059472384,2020-03-31,RT @akagiichirou: マスクの７割が中国からの輸入に頼ってきたが流通網が混乱す...,True,2020,3,31,23,23:59:58
4,128360620,2020-03-31,RT @bbcnewsjapan: BBCニュース - ハンガリー政府、新型ウイルス対策で強...,True,2020,3,31,23,23:59:57
